## Import Libraries

In [16]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [17]:
import matplotlib.pyplot as plt
plt.ion()

In [18]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [19]:
from os import path, pardir

## Compile and program project

In [20]:
project_path = [path.curdir, "FFT"]
project_path

['.', 'FFT']

In [22]:
atprogram("./FFT", device_name = "ATSAML21J18B",verbose=3)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:54100
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x000056d0.
[DEBUG] Memory segment base written at 0x000056d0. Size = 0x00000068.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Starting execution of "info"
[ERROR] Error in TCF lockbit format. Missed key Could not get Module LOCKBIT for ATSAML21J18B. (TCF Error code: 131120)
[DEBUG] Command "info" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.
Tool edbg has firmware version: 03.25
Target voltage: 3.31 V

Device information:

Name:       ATSAML21J18B 

0

## Data Logging

In [23]:
live_plot = True

Create a figure for the plot.

In [24]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [25]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_fft"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment FFT",
}

Stop criteria to pass to the logger:

In [26]:
def stop_fn(logger_data):
    return len(logger_data.gpio) and all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [27]:
def stop_function(pin_values):
    return all(pin_values)

In [28]:
repetitions = 10

Perform the measurement.

In [29]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
        
charges_allocate = []
charges_fft = []
charges_free = []
times_allocate = []
times_fft = []
times_free = []


with DGILibExtra(**cd) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(10,stop_fn)
        charge2, time2 = power_and_time_per_pulse(dgilib.data, 2, stop_function=stop_function)
        charge3, time3 = power_and_time_per_pulse(dgilib.data, 3, stop_function=stop_function)
        
        if len(charge2) != 2 or len(time2) != 2 or len(charge3) != 1 or len(time3) != 1:
            print(f"Parsing of measurement data failed, charge2:{charge2}, time2:{time2}, charge3:{charge3}, time3:{time3},")
            break
        charges_allocate.append(charge2[0])
        charges_fft.append(charge3[0])
        charges_free.append(charge2[1])
        times_allocate.append(time2[0])
        times_fft.append(time3[0])
        times_free.append(time2[1])
        dgilib.empty_data()

    charges = charges_allocate + charges_fft + charges_free 
    times = times_allocate + times_fft + times_free
    
    
    charges = [charge_allocate + charge_fft + charge_free for charge_allocate, charge_fft, charge_free in zip(charges_allocate, charges_fft, charges_free)]
    times = [time_allocate + time_fft + time_free for time_allocate, time_fft, time_free in zip(times_allocate, times_fft, times_free)]

    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

In [30]:
import json

config = {}
config["name"] = "FFT 128"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Allocate FFT", "Free FFT"], 3: ["FFT"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": ["Allocate FFT"], 
                                        "loop": ["FFT"],
                                        "exit": ["Free FFT"]}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

In [31]:
import pandas as pd

In [32]:
df = pd.DataFrame({'charges_allocate': charges_allocate, 'charges_fft':charges_fft, 'charges_free':charges_free, 'times_allocate':times_allocate, 'times_fft':times_fft, 'times_free':times_free})

In [33]:
df

,charges_allocate,charges_fft,charges_free,times_allocate,times_fft,times_free
0,0.000314,0.000072,2.612489e-08,0.488990,0.125926,0.000066
1,0.000314,0.000072,3.358995e-08,0.488992,0.125926,0.000066
2,0.000314,0.000072,0.000000e+00,0.488998,0.125995,0.000000
3,0.000314,0.000072,0.000000e+00,0.489130,0.125995,0.000000
4,0.000314,0.000072,2.612489e-08,0.489069,0.125925,0.000066
5,0.000314,0.000072,2.519186e-08,0.489064,0.125929,0.000066
6,0.000314,0.000072,2.612531e-08,0.489057,0.125925,0.000066
7,0.000314,0.000072,2.519450e-08,0.489056,0.125929,0.000066
8,0.000314,0.000072,2.612531e-08,0.489000,0.125929,0.000066
9,0.000314,0.000072,0.000000e+00,0.488998,0.125930,0.000000


In [34]:
df.describe()

,charges_allocate,charges_fft,charges_free,times_allocate,times_fft,times_free
count,1.000000e+01,1.000000e+01,1.000000e+01,10.000000,10.000000,10.000000
mean,3.136024e-04,7.224295e-05,1.884767e-08,0.489035,0.125941,0.000046
std,6.921281e-08,6.023594e-08,1.323049e-08,0.000047,0.000029,0.000032
min,3.135148e-04,7.215042e-05,0.000000e+00,0.488990,0.125925,0.000000
25%,3.135442e-04,7.220244e-05,6.297964e-09,0.488998,0.125926,0.000016
50%,3.135947e-04,7.225767e-05,2.565969e-08,0.489028,0.125929,0.000066
75%,3.136452e-04,7.226577e-05,2.612520e-08,0.489062,0.125930,0.000066
max,3.137169e-04,7.233903e-05,3.358995e-08,0.489130,0.125995,0.000066
